<a href="https://colab.research.google.com/github/nashrah-18/YourIndianTranslatorv1/blob/main/FinalEngToHin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Checking if GPU is running or not
#!nvidia-smi

In [2]:
#LETS FIRST PUSH MY CREATED DATASET INTO MY HUGGINGFACE ACCOUNT
!pip install datasets huggingface_hub

In [13]:

import tensorflow as tf
import json

# Load JSON data
with open('/content/eng-hindi.json', 'r') as f:
    data = json.load(f)

# Create a generator
def generator():
    for row in data:
        yield row

# Define the output types
output_types = {
    'english_text': tf.string,
    'hindi_translation': tf.string,
    'topic': tf.string
}

# Convert to TensorFlow Dataset
dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=output_types
)

# Print the 5th row (index 4)
for i, row in enumerate(dataset):
    if i == 4:
        print(row)
        break

{'english_text': <tf.Tensor: shape=(), dtype=string, numpy=b'How are you?'>, 'hindi_translation': <tf.Tensor: shape=(), dtype=string, numpy=b'\xe0\xa4\x95\xe0\xa5\x8d\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\xb9\xe0\xa4\xbe\xe0\xa4\xb2 \xe0\xa4\x9a\xe0\xa4\xbe\xe0\xa4\xb2 \xe0\xa4\xb9\xe0\xa5\x88?'>, 'topic': <tf.Tensor: shape=(), dtype=string, numpy=b'Language Basics and Phrases'>}


In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
#DATA TOKENIZING AND MODEL TRAINING
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.3 MB/s eta 0:00:00


In [19]:
#Imports
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
#Choose the model
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [ ]:
#Push dataset
dataset.push_to_hub("nashrah18/indiantranslator")

In [ ]:
#Load our dataset that we just pushed so we can train it
raw_datasets = load_dataset("nashrah18/indiantranslator")

README.md:   0%|          | 0.00/493 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/236 [00:00<?, ? examples/s]

In [ ]:
#Check the dataset
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['english_text', 'hindi_translation', 'topic'],
        num_rows: 236
    })
})

In [ ]:
#Check specific column
raw_datasets['train'][100]

{'english_text': 'City',
 'hindi_translation': 'शहर',
 'topic': 'Language Basics and Phrases'}

In [ ]:
#PREPROCESSING OUR DATA
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
#Check tokenizer
tokenizer("Hello, this is a sentence!")

In [ ]:
#Checking two sentences at the same time
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 23, 19, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
#Check whether it does tokenizing to hindi sentences
with tokenizer.as_target_tokenizer():
    print(tokenizer(["यार, इंडिया में टूरिस्ट्स पर आम ऑनलाइन स्कैम्स कौन से हैं?"]))

{'input_ids': [[13005, 2, 28144, 11, 44122, 10972, 1307, 33, 2152, 4871, 18784, 16284, 1307, 391, 12, 28, 22, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
#Convert it into numbers
max_input_length=128
max_target_length=128

source_lang="english_text" #your dataset column name
target_lang="hindi_translation" #your dataset column name

def preprocess_function(examples):
    # Access the correct columns using column names
    inputs = examples[source_lang]
    targets = examples[target_lang]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[12110, 0], [172, 16, 213, 22, 0]], 'attention_mask': [[1, 1], [1, 1, 1, 1, 1]], 'labels': [[4788, 351, 26090, 0], [65, 1218, 1110, 22, 0]]}

In [ ]:
#Map tokens
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [ ]:
#Load model
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
learning_rate = 5e-5
weight_decay = 0.01
num_train_epochs = 15 #keep epochs high

In [ ]:
#Passes data to your model batch wise
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
#Passes data with parameters
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
#Load Training data
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
#Compile your model
#Deploy your optimizer
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
#Feed my dataset to my model
model.fit(train_dataset, epochs=15)

Epoch 1/15
14/14 [==============================] - 135s 7s/step - loss: 2.7680
Epoch 2/15
14/14 [==============================] - 99s 7s/step - loss: 1.7672
Epoch 3/15
14/14 [==============================] - 94s 7s/step - loss: 1.2098
Epoch 4/15
14/14 [==============================] - 95s 7s/step - loss: 0.9267
Epoch 5/15
14/14 [==============================] - 95s 7s/step - loss: 0.6953
Epoch 6/15
14/14 [==============================] - 91s 7s/step - loss: 0.5534
Epoch 7/15
14/14 [==============================] - 96s 7s/step - loss: 0.4267
Epoch 8/15
14/14 [==============================] - 93s 7s/step - loss: 0.3309
Epoch 9/15
14/14 [==============================] - 92s 7s/step - loss: 0.2949
Epoch 10/15
14/14 [==============================] - 92s 7s/step - loss: 0.2394
Epoch 11/15
14/14 [==============================] - 106s 8s/step - loss: 0.2157
Epoch 12/15
14/14 [==============================] - 92s 7s/step - loss: 0.1717
Epoch 13/15
14/14 [============================

In [ ]:
#Save the model
model.save_pretrained("indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


In [ ]:
#Tokenize model
tokenizer.save_pretrained("indian_translatorv1")

('indian_translatorv1/tokenizer_config.json',
 'indian_translatorv1/special_tokens_map.json',
 'indian_translatorv1/vocab.json',
 'indian_translatorv1/source.spm',
 'indian_translatorv1/target.spm',
 'indian_translatorv1/added_tokens.json')

In [ ]:
#Push our model indian_translatorv1
model.push_to_hub("nashrah18/indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [ ]:
#Tokenize our model
tokenizer.push_to_hub("nashrah18/indian_translatorv1")

README.md:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nashrah18/indian_translatorv1/commit/917a1a809ea9a343c76807ae5c50374c1a570096', commit_message='Upload tokenizer', commit_description='', oid='917a1a809ea9a343c76807ae5c50374c1a570096', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nashrah18/indian_translatorv1', endpoint='https://huggingface.co', repo_type='model', repo_id='nashrah18/indian_translatorv1'), pr_revision=None, pr_num=None)

#MODEL TESTING

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("indian_translatorv1")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at indian_translatorv1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
#Tokenize output
input_text  = "Brother"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
#Change numbers into text
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

भाईसाहब


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from IPython.display import HTML, display
from ipywidgets import interact, widgets

def interactive_translation(text_input):
    if text_input:
        inputs = tokenizer([text_input], return_tensors='np')
        out = model.generate(**inputs, max_length=128)
        hindi_text = tokenizer.decode(out[0], skip_special_tokens=True)
        display(HTML(f"""
            <div style='padding: 10px; border-radius: 5px;'>
                <b>English:</b> {text_input}<br>
                <b>Hindi:</b> {hindi_text}
            </div>
        """))

# Create interactive widget
interact(interactive_translation, text_input=widgets.Text(description='English:', placeholder='Enter text to translate'))

interactive(children=(Text(value='', description='English:', placeholder='Enter text to translate'), Output())…

<function __main__.interactive_translation(text_input)>